# Packing CAT Cell demo notebook
The main functions of this demo are located in :
- [chordal_axis_transform.py](../irregular_object_packing/packing/chordal_axis_transform.py)
- [nlc_optimisation.py](../irregular_object_packing/packing/nlc_optimisation.py)


### Contents
1. 
2. 

### Pre-requisites
Please install the required packages using the following command:

```bash
pip install -r requirements.txt
```

# [WARNING] This demo is not correct anymore. I recommend to take a look at the ./demo-packing.ipynb notebook.



In [1]:
# IMPORTS
import sys
sys.path.append('../')

import numpy as np
import pyvista as pv
import trimesh

from irregular_object_packing.mesh.utils import print_mesh_info
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume, translation_matrix
import irregular_object_packing.packing.chordal_axis_transform as cat
import irregular_object_packing.packing.nlc_optimisation as nlc
from irregular_object_packing.packing.initialize import init_coordinates

## Basic CAT Cell demo
The cell below shows a simple demo of computing cat faces. The container is a cube and the object os a cuboid rotated 45 degrees over the z axis. The yellow faces are the CAT faces.

In [2]:
from irregular_object_packing.packing.chordal_axis_transform import main
main()

/opt/homebrew/anaconda3/envs/taichi310/lib/python3.10/site-packages/pyvista/core/dataset.py:2668: PyVistaDeprecationWarning: `cell_points` is deprecated. Use `cell[i].points` instead
  warnings.warn(


Widget(value="<iframe src='http://localhost:49649/index.html?ui=P_0x15d107940_0&reconnect=auto' style='width: …

Widget(value="<iframe src='http://localhost:49649/index.html?ui=P_0x15d107850_1&reconnect=auto' style='width: …

## Demo computing CAT cells for Blood Cells in a cylindrical container
The cell below shows a demo of computing cat faces for a cylindrical container and multiple blood cell meshes. The yellow faces are the CAT faces and for each red blood cell there is a corresponding CAT cell. 



In [3]:
mesh_volume = 0.1
container_volume = 10
coverage_rate = 0.3

DATA_FOLDER = "../data/mesh/"
loaded_mesh = pv.read(DATA_FOLDER + "RBC_normal.stl")
print_mesh_info(loaded_mesh, "loaded mesh")
# trimesh.Scene([loaded_mesh]).show()
# Scale the mesh to the desired volume
original_mesh = scale_and_center_mesh(loaded_mesh, mesh_volume)
print_mesh_info(original_mesh, "scaled mesh")

container = pv.Sphere(radius=1, center=[0,0,0])
# container = trimesh.primitives.Cylinder(radius=1, height=1)
print_mesh_info(container, "original container")

container = scale_to_volume(container, container_volume)
print_mesh_info(container, "scaled container")

Mesh info loaded mesh: PolyData (0x15d548100)
  N Cells:    1280
  N Points:   642
  N Strips:   0
  X Bounds:   3.509e+01, 4.291e+01
  Y Bounds:   3.796e+01, 4.004e+01
  Z Bounds:   3.509e+01, 4.291e+01
  N Arrays:   0
, 
volume: 71.04896301964186, 
bounding box: (35.0890007019043, 42.9109992980957, 37.96297836303711, 40.03702163696289, 35.0890007019043, 42.9109992980957) 
center of mass: [39. 39. 39.]

Mesh info scaled mesh: PolyData (0x15d548160)
  N Cells:    1280
  N Points:   642
  N Strips:   0
  X Bounds:   -4.383e-01, 4.383e-01
  Y Bounds:   -1.162e-01, 1.162e-01
  Z Bounds:   -4.383e-01, 4.383e-01
  N Arrays:   0
, 
volume: 0.10000000835585507, 
bounding box: (-0.4382968842983246, 0.43829718232154846, -0.11621677875518799, 0.11621654033660889, -0.438296914100647, 0.4382971525192261) 
center of mass: [ 0.  0. -0.]

Mesh info original container: PolyData (0x15d5488e0)
  N Cells:    1680
  N Points:   842
  N Strips:   0
  X Bounds:   -9.985e-01, 9.985e-01
  Y Bounds:   -9.931e-

In [4]:
# Initial placement of the objects
objects_coords, _ = init_coordinates(container, original_mesh, coverage_rate=coverage_rate)


In [5]:
# get vertices of the object meshes and the container
# we resample to simplify and get a more uniform distribution of points
from irregular_object_packing.mesh.utils import resample_pyvista_mesh, resample_pyvista_mesh_kmeans

mesh = resample_pyvista_mesh(original_mesh, 200)
down_scale = 0.1 # initial downscale 
obj_points = []
tf_matrices = []
for i in range(len(objects_coords)):
    # get the object
    object = mesh.copy()
    # random rotation
    M_rot = trimesh.transformations.random_rotation_matrix()
    M_f = trimesh.transformations.scale_matrix(down_scale**(1/3))
    M_t = trimesh.transformations.translation_matrix(objects_coords[i] - np.array([0, 0, 0]))

    M = M_t @ M_f @ M_rot
    # first scale, then rotate, then translate
    points = object.transform(M, inplace=False).points
    # print(points.shape)

    tf_matrices.append(M)
    obj_points.append(points)

container_points = resample_pyvista_mesh_kmeans(container, 200)

PolyData (0x15d50fd00)
  N Cells:    200
  N Points:   102
  N Strips:   0
  X Bounds:   -4.357e-01, 4.357e-01
  Y Bounds:   -1.168e-01, 1.168e-01
  Z Bounds:   -4.365e-01, 4.362e-01
  N Arrays:   0



In [6]:
# compute the cat cells for each object (takes approx. 1 min)
cat_data = cat.compute_cat_cells(obj_points, container_points.points, objects_coords)
cat_data.object_coords = objects_coords


/opt/homebrew/anaconda3/envs/taichi310/lib/python3.10/site-packages/pyvista/core/dataset.py:2668: PyVistaDeprecationWarning: `cell_points` is deprecated. Use `cell[i].points` instead
  warnings.warn(


In [7]:
# reload(cat); reload(nlc)
from scipy.optimize import minimize
# for object 0:
    # Define the bounds for the variables
obj_id = 0

def optimal_local_transform(
    obj_id, cat_data, scale_bound=(0.1, None), max_angle=1 / 12 * np.pi, max_t=None, margin=None
):
    """Computes the optimal local transform for a given object id. This will return the transformation parameters that
    maximises scale with respect to a local coordinate system of the object. This is possible due to the `obj_coords`.
    """

    r_bound = (-max_angle, max_angle)
    t_bound = (0, max_t)
    bounds = [scale_bound, r_bound, r_bound, r_bound, t_bound, t_bound, t_bound]
    x0 = np.array([scale_bound[0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

    constraint_dict = {
        "type": "ineq",
        "fun": nlc.local_constraints_from_cat,
        "args": (
            obj_id,
            cat_data,
            margin,
        ),
    }
    res = minimize(nlc.objective, x0, method="SLSQP", bounds=bounds, constraints=constraint_dict)
    return res.x

tf_arr = optimal_local_transform(obj_id, cat_data, scale_bound=(0.1, None), max_angle=1 / 12 * np.pi, max_t=None)


In [8]:
# This next part of code is for the visualization of the CAT cells.
# it processes the output of compute_cat_cells to create a PyVista mesh for each object
# and it applies the corresponding transformations on the object meshes.
from irregular_object_packing.packing.chordal_axis_transform import face_coord_to_points_and_faces
from irregular_object_packing.packing.nlc_optimisation import construct_transform_matrix

mesh_before = mesh.transform(tf_matrices[obj_id], inplace=False)
mesh_after = mesh.transform(construct_transform_matrix(tf_arr), inplace=False)
cat_mesh = pv.PolyData(*face_coord_to_points_and_faces(cat_data, obj_id))


In [9]:
from irregular_object_packing.packing.plots import plot_step_comparison

plotter = plot_step_comparison(mesh_before, mesh_after, cat_mesh)

print("This demo is not correct anymore. I recommend to take a look at the ./demo-packing.ipynb notebook.")

Widget(value="<iframe src='http://localhost:49649/index.html?ui=P_0x15e142ef0_3&reconnect=auto' style='width: …